# Pangenomics
--------------------------------------------

# Variant Calling with vg

## Overview

Variants can be called both within the pangenomic graph and by aligning reads to the graph. You will learn how to call variants both ways in this submodule.

## Learning Objectives
+ Understand different types of variants
+ Understand our ability to call variants with different types of reads and pangenomic graphs
+ Learn how to call and interpret variants with VG

## Get Started

When calling variants, we use aligned reads to find support for variants contained in the graph. For the original pangenome graph, it will find variants from the assemblies used to make the graph. You can also augment the pangenome graph with novel variants in the reads, creating an augemented pangenome graph that can be used to call variants.

First we will learn how to identify variants that are supported by the graph. Then we'll look at identifying novel variants that are not in the graphs.

#### Variant Calling
- Variants supported by the graph
- Novel variants

----------------------

## Call Variants

We will look for two variant types:</mark
- Variants that are supported by the graph.
- Variants that are novel (i.e., not in the graph but supported by the reads aligned to the graph).

We will call variants against the graph, though you could also call variants using the surjected BAM file and traditional variant calling methods.

----------------------

## Calling Graph Supported Variants

1. Compute read support for variation already in the graph using `vg pack`.

The parameters:

`-x`  the graph 
`-g`  aligments in gam format  
`-Q`  ignore mapping and base qualities < N  
`-s`  ignore the first and last N nucleotides of each read  
`-o`  the output pack file  
`-t`  use N threads

In [ ]:
!vg pack -x graphs/yprp.chrVIII.pggb.giraffe.gbz -g alignments/SK1xyprp.chrVIII.pggb.mapped.gam -Q 5 -s 5 -t 4 -o alignments/yprp.chrVIII.pggb.mapped.pack

2. Generate a VCF from the read support using `vg call`.

The parameters:

`-k`  The read support file to read in  
`-t`  The number of threads
`-z`  Restrict the search to GBZ haplotypes (can improve speed and accuracy); we won't use this here

Also, feed in the graph as a positional argument:  
*graphs/yprp.chrVIII.pggb.giraffe.gbz*

In [ ]:
!vg call -k alignments/yprp.chrVIII.pggb.mapped.pack -t 4 graphs/yprp.chrVIII.pggb.giraffe.gbz > variants/SK1xyprp.chrVIII.pggb.graph_calls.vcf

<div class="alert alert-block alert-success"> <b>Try this in the cell below:</b>  
    <ul>
        <li>Call variants in the full genome graph by computing read support and then generating a vcf file.</li>
    </ul>

In [ ]:
# Call variants in the full genome graph

<details>
<summary>Click for help</summary>
<br>
!vg pack -x graphs/yprp.fullgenome.pggb.giraffe.gbz -g alignments/SK1xyprp.fullgenome.pggb.mapped.gam -Q 5 -s 5 -o alignments/yprp.fullgenome.pggb.mapped.pack -t 4   


!vg call -k alignments/yprp.fullgenome.pggb.mapped.pack -t 4 graphs/yprp.fullgenome.pggb.giraffe.gbz > variants/yprp.fullgenome.pggb.graph_calls.vcf
</details>

----------------------

## Including Novel Variant Calls

1. To call novel variants (i.e., those variants supported by the aligned reads),  we need to embed the variation from the reads we aligned back into the graph. To do this we need to convert the graph into a form that we can change. We will use `vg convert` to convert the .gbz file to a .vg file.

In [ ]:
!vg convert graphs/yprp.chrVIII.pggb.giraffe.gbz > graphs/yprp.chrVIII.pggb.giraffe.vg

2. Now, we can augment the graph with the mapped reads using `vg augment`. This will embed the variation from the alignments back into the graph.

The parameters:

`-A`  new, augmented graph with aligned reads  
`-t`  the number of threads to use  

Also, feed in the the graph and the input alignment (gam) file as positional arguments:  
*graphs/yprp.chrVIII.pggb.giraffe.vg*  
*alignments/SK1xyprp.chrVIII.pggb.mapped.gam*

In [ ]:
!vg augment graphs/yprp.chrVIII.pggb.giraffe.vg alignments/SK1xyprp.chrVIII.pggb.mapped.gam -A alignments/SK1xyprp.chrVIII.pggb.mapped.aug.gam -t 4 > graphs/SK1xyprp.chrVIII.pggb.aug.vg 

3. Next, index the augmented graph using `vg index`. We will make a .xg index.

The parameters:

`-x`  output file  
`-t`  the number of threads  

Also, feed in the graph as a positional argument:  
*graphs/SK1xyprp.chrVIII.pggb.aug.vg*

In [ ]:
!vg index -t 4 -x graphs/SK1xyprp.chrVIII.pggb.aug.xg graphs/SK1xyprp.chrVIII.pggb.aug.vg

4. Now that the variation from the reads is embedded into the graph along with the original variants, we can procede to call variants like we did above by computing read support as we show below. 

In [ ]:
!vg pack -x graphs/SK1xyprp.chrVIII.pggb.aug.xg -g alignments/SK1xyprp.chrVIII.pggb.mapped.aug.gam -Q 5 -s 5 -o alignments/SK1xyprp.chrVIII.pggb.mapped.aug.pack -t 4

5. Then, we generate a VCF from the support.

In [ ]:
!vg call graphs/SK1xyprp.chrVIII.pggb.aug.xg -k alignments/SK1xyprp.chrVIII.pggb.mapped.aug.pack -t 4 > variants/SK1xyprp.chrVIII.pggb.aug_calls.vcf

6. Finally, we generate stats on this VCF file. We will use `grep` to pull out the rows that start with SN.

In [ ]:
!bcftools stats variants/SK1xyprp.chrVIII.pggb.aug_calls.vcf | grep "^SN"

SNPs = single nucleotide polymorphisms (a single nucleotide change; reference and alternate alleles are all of length 1)  
MNPs = multi-nucleotide polymorphisms (reference and alternate alleles are all of the same length and that length is >1)  
indels = insertion/deletion (reference and alternate alleles are of different lengths)
others = more complex variants
multiallelic sites = more than one alternate allele
multiallelic SNP sites = more than one alternate allele at a SNP site

<div class="alert alert-block alert-success"> <b>Try this in the cells below:</b>  
    <ul>
        Call novel variants for the full genome graph (*yprp.fullgenome.pggb.giraffe.gbz*) by performing the following steps:
        <li>Convert the graph to vg format.</li>
        <li>Augment the graph to embed the read alignments.</li>
        <li>Create an index (xg).</li>
        <li>Compute read support.</li>
        <li>Generate a VCF.</li>
        <li>Generate statistics.</li>
    </ul>

In [ ]:
# Convert the graph to vg format

In [ ]:
# Augment the graph to embed the read alignments

In [ ]:
# Create an index (xg)

In [ ]:
# Compute read support

In [ ]:
# Generate a VCF

In [ ]:
# Generate statistics

<details>
<summary>Click for help</summary>
<br>
    
!vg convert graphs/yprp.fullgenome.pggb.giraffe.gbz > graphs/yprp.fullgenome.pggb.giraffe.vg 

!vg augment graphs/yprp.fullgenome.pggb.giraffe.vg alignments/SK1xyprp.fullgenome.pggb.mapped.gam -A alignments/SK1xyprp.fullgenome.pggb.mapped.aug.gam -t 4 > graphs/SK1xyprp.fullgenome.pggb.aug.vg 

!vg index -t 4 -x graphs/SK1xyprp.fullgenome.pggb.aug.xg graphs/SK1xyprp.fullgenome.pggb.aug.vg

!vg pack -x graphs/SK1xyprp.fullgenome.pggb.aug.xg -g alignments/SK1xyprp.fullgenome.pggb.mapped.aug.gam -Q 5 -s 5 -o alignments/SK1xyprp.fullgenome.pggb.mapped.aug.pack -t 4

!vg call graphs/SK1xyprp.fullgenome.pggb.aug.xg -k alignments/SK1xyprp.fullgenome.pggb.mapped.aug.pack -t 4 > variants/SK1xyprp.fullgenome.pggb.aug_calls.vcf

!bcftools stats alignments/SK1xyprp.fullgenome.pggb.aug_calls.vcf | grep "^SN"

</details>

In [ ]:
----------------------

## Variants used to construct the graph

We can also pull out variants that are in the structure of the graph, which does not require aligning reads from a sample.

1. First, we will create a .xg formated graph with `vg index`.

In [ ]:
!vg index -t 4 -x graphs/yprp.chrVIII.pggb.xg graphs/yprp.chrVIII.pggb.vg

2. Now, grab the variants from the graph using `vg deconstruct`.

The parameters:
  
`-P` path prefix (variants will be called for all paths that start with this prefix)  
`-t` number of threads

Also, feed in the graph as a positional argument:  
*Graph (xg)*

<div class="alert alert-block alert-info"> <b>NOTE:</b> VG takes liberties with variants when constructing the graph so this VCF might not be identical to those from other methods.

In [ ]:
!vg deconstruct graphs/yprp.chrVIII.pggb.xg -P S288C -t 20 > variants/yprp.chrVIII.pggb.S288Cpaths.deconstruct.vcf

<div class="alert alert-block alert-sucess"> <b>Try this in the cell below:</b>  
    <ul>
        <li>Deconstruct variants from the full genome graph (yprp.fullgenome.pggb.giraffe.vg)</li>
        <li>Hint: first you will need to create an .xg version using `vg index`</li>
    </ul>

In [ ]:
# Deconstruct variants from the full genome graph

<details>
<summary>Click for help</summary>
<br>
!vg index -t 4 -x graphs/yprp.fullgenome.pggb.xg graphs/yprp.fullgenome.pggb.vg   


!vg deconstruct graphs/yprp.fullgenome.pggb.xg -P S288C -t 20 > variants/yprp.fullgenome.pggb.S288Cpaths.deconstruct.vcf
</details>

----------------------

## Visualize the original and augmented graphs

1. Compare CUP1 region of the original graph (yprp.chrVIII.pggb.gfa) and augmented graph (SK1xyprp.chrVIII.pggb.aug.vg, which we will convert to SK1xyprp.chrVIII.pggb.aug.gfa) in bandage. First, convert the augmented graph to gfa format.

The parameters:

`-f`  output in GFA format

Also, feed in the graph as a positional argument:  
*graphs/SK1xyprp.chrVIII.pggb.aug.vg*

In [ ]:
!vg convert -f graphs/SK1xyprp.chrVIII.pggb.aug.vg > graphs/SK1xyprp.chrVIII.pggb.aug.gfa

2. Then visualize the CUP1 region of each .gfa file in bandage.

<div class="alert alert-block alert-info"> <b>NOTE:</b> The augmented graph is much bigger and it will have difficulty loading the entire chrVIII graph. So, before drawing the graph, blast the genes. Change the "Scope" to "Around query hits". Change "Distance" to 200 for the augmented graph. Then click "Draw Graph".  


<div class="alert alert-block alert-info"> <b>Questions:</b>  
    <ul>
        <li>What differences do you see between the CUP1 region in both graphs?</li>
        <li>How do the number of nodes, number of edges, and the total length differ between the original and the augmented graph?</li>
    </ul>

<details>
<summary>Click for possible answers</summary>
<br>
CUP1 region from the original graph.  

![CUP1 region](./Figures/CUP1region.png)

CUP1 region from the augmented graph.

![CUP1 region](./Figures/auggraph.png)

These graphs are actually fairly similar except the augment graph is much more broken up, making the graph look like it has more "dotted" lines. There are small differences in the loops, especially the double loop near the 2 straight ends.


|        | Original | Augmented |
|--------|----------|-----------|
| Nodes  | 18,856   | 590,574   |
| Edges  | 25,371   | 886,845   |
| Length | 623,013  | 938,699   |


The augmented graph is broken into many more pieces. This is expected given that adding in variation breaks nodes. It is also much longer, capturing more genetic variation.
</details>

----------------------

## Conclusion

In this submodule, you learned different ways to call and characterize variants from the graph, including variants supported within the graph and novel variants supported by reads that were used to augment the graph.

----------------------

## Module Review

Congratulations, you have completed the pangenomics module!

In this module, you learned about pangenomics and used a yeast dataset to build pangenomics graphs using PGGB. You learned how to search these graphs for regions that match DNA sequence queries using BLAST and how to interactively visualize these graphs using Bandage. In addition, you learned how to index the graphs, map reads to the graphs, and call variants. Well done!

----------------------

## Clean up

<div class="alert alert-warning">No cleanup is necessary for this submodule. Don't forget to shutdown your Workbench when you are done working through this module!  

----------------------
    
If you do not plan to come back to review this module or to try to run your own data in this environment, delete your Workbench so you do not continue to incur charges.

</div>